In [1]:
from bs4 import BeautifulSoup
import os
import re
import csvtools
import pandas as pd
from nltk import word_tokenize
import string
import pymorphy2
import bs4
import collections
from collections import Counter
import nltk
import nltk.tokenize.punkt
import requests
from datetime import datetime
from prozhitotools import dump
from tqdm.auto import tqdm
import ipywidgets
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF
morph = pymorphy2.MorphAnalyzer()
extended_punctuation = string.punctuation + '\'\'" №~—»«..``’.--–“”―…"''" '
from stop_words import get_stop_words
stoplist = get_stop_words('ru') + ['среди', 'аа', 'некий', 'либо', 'видимо', 'итак', 'словно', 'наверное', 'возможно', '-й', '-х', 'например', 'что-нибудь', 'вместо', 'кто-то', 'несмотря', 'коли', 'безъ', 'нѣтъ', 'всѣхъ', 'поэтому', 'почему-то', 'ибо', 'из-за', 'можетъ', 'неужели', 'дѣть', 'намъ', 'оттого', 'чрез', 'вероятно', 'многий', 'ах', 'пред', 'пусть', 'нынче', 'б', 'дабы', 'чрезвычайно', 'разумеется', 'вовсе', 'кн', 'гр', 'св', '-го', 'кой', 'франц.', 'оный', 'сей', 'свой', 'едва', 'самый', 'го', 'мол', 'час', 'часы', 'ть', 'минута', 'метр', 'г.', 'ко', 'въ', 'съ', 'мнѣ', 'къ', 'какъ', 'отъ', 'такъ', 'ежели', 'себѣ', 'былъ', 'онъ', 'томъ', 'сие', 'нрзб', 'изъ', 'тѣмъ', 'ихъ', 'чѣмъ', 'всѣ'] + ['com', 'id', 'de', 'la', 'et', 'le', 'que', 'je', 'il', 'est', 'les', 'un']

In [2]:
import sys
import wheel
#!{sys.executable} -m pip install python-Levenshtein
#!pip install python-Levenshtein

In [520]:
#import getpass
#import os

In [522]:
#password = getpass.getpass()
#command = "!sudo rm /usr/local/lib/pkcs11/librtpkcs11ecp.dylib"
#os.popen(command, 'w').write(password+'\n')

 ············


13

In [3]:
dw = dump.Wrapper(csvpath='/Users/victoriavrbv/Downloads/prozhito-dump-1')

In [4]:
authors_data = pd.read_csv('/Users/victoriavrbv/Downloads/Prozhito-people - person_data.csv')

In [5]:
authors_data

,id,first_name,patronymic,family_name,gender,birth_date,death_date,edition,additional_info,from_natasha,from_tags,info,nickname,wiki
0,3739,Чудинова,Александра,Дмитриевна,NaN,NaN,NaN,NaN,NaN,False,True,NaN,NaN,NaN
1,3740,Аборкин,Дмитрий,Георгиевич,NaN,NaN,NaN,NaN,NaN,False,True,NaN,NaN,NaN
2,3741,ТИХОНОВ,СЕРГЕЙ,Константинович,NaN,NaN,NaN,NaN,NaN,False,True,NaN,NaN,NaN
3,3742,Митрофанов,Владимир,Петрович,NaN,NaN,NaN,NaN,NaN,False,True,NaN,NaN,NaN
4,3743,Манский,Давыд,Моисеевич,NaN,NaN,NaN,NaN,NaN,False,True,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28980,28808,с,.,найденов,NaN,NaN,NaN,NaN,NaN,True,False,NaN,NaN,NaN
28981,28874,Иоганна,Георга,Корба,NaN,NaN,NaN,NaN,NaN,True,False,NaN,NaN,NaN
28982,28809,л,.,авилова,NaN,NaN,NaN,NaN,NaN,True,False,NaN,NaN,NaN
28983,29003,де,-,жимель,NaN,NaN,NaN,NaN,NaN,True,False,NaN,NaN,NaN


In [93]:
#authors_data[(authors_data["from_natasha"] == False) & (authors_data["from_tags"] == False)]
#authors_data[authors_data.edition.notnull()|authors_data["from_natasha"] == False]

In [6]:
#считаем сколько мужчин и женщин в базе
authors_data.groupby('gender')['id'].nunique()

gender
0.0     810
1.0    2882
Name: id, dtype: int64

In [381]:
counter = 0
for item in authors_data.edition[authors_data.edition.notnull()]:
    if re.search("^(?!.*\*\*Издани(е|я):\s?\*\*).*$", str(item)):
        counter+=1
counter

40

In [6]:
#функция для лемматизации
def preprocess_text(some_text):
    '''
    берет обычный текст, возвращает лемматизированный
    '''    
    some_text = re.sub(r'\[|\]|\\|([0-9])|[a-zA-ZàâäôéèëêïîçùûüÿæœÀÂÄÔÉÈËÊÏÎŸÇÙÛÜÆŒ~]', '', str(some_text))
    lemmatized_text = [morph.parse(word)[0].normal_form for word in word_tokenize(some_text.lower()) if
                       morph.parse(word)[0].normal_form not in stoplist and morph.parse(word)[0].normal_form not in extended_punctuation]
    return lemmatized_text

In [37]:
stoplist2 = get_stop_words('ru') + ['фр', 'франц.', 'оный', 'сей', 'т.е', 'свой', 'едва', 'самый', 'го', 'мол', 'час', 'часы', 'ть', 'минута', 'метр', 'г.', 'ко', 'въ', 'съ', 'мнѣ', 'къ', 'какъ', 'отъ', 'такъ', 'ежели', 'себѣ', 'былъ', 'онъ', 'томъ', 'сие', 'нрзб', 'изъ', 'тѣмъ', 'ихъ', 'чѣмъ', 'всѣ'] + ['com', 'id', 'de', 'la', 'et', 'le', 'que', 'je', 'il', 'est', 'les', 'un']
names_list = ["владимир", "аля", "лёля", "ася", "ник", "никола", "юрий", "борис", "виктор", "андрей", "ирина", "варя", "валя", "анович", "шура", "боря", "шурочка", "леонид", "вера", "зуля", "зося", "михаилович", "николаевич", "наташа", "митя", "ляля", "иванович", "зина", "володя", "никита", "лена", "коля", "николай", "маруся", "лиза", "олег", "вася", "игорь", "георгий", "львович", "марина", "софья", "андреевич", "александр", "цветаев", "владимирович", "таня", "анна", "василиевич", "петрович", "сергей", "василиевич", "саша", "павлович", "маша", "миша", "константин", "пётр", "павел", "лена", "лев", "серёжа", "оля", "яковлевич", "нина", "шурик", "женя", "соня", "семён", "костя"]
def preprocess_text_for_topics(some_text):
    '''
    берет обычный текст, возвращает лемматизированный
    '''    
    some_text = re.sub(r'\[|\]|\\|([0-9])|[a-zA-ZàâäôéèëêïîçùûüÿæœÀÂÄÔÉÈËÊÏÎŸÇÙÛÜÆŒ~]', '', str(some_text))
    lemmatized_text = [morph.parse(word)[0].normal_form for word in word_tokenize(some_text.lower()) if
                       morph.parse(word)[0].normal_form not in stoplist2 and morph.parse(word)[0].normal_form not in extended_punctuation]
    return lemmatized_text

In [9]:
len(dw.notes)

384587

In [153]:
#отделяем записи 17-19 века
war_records = []
period = dw.notes[(1942, 1, 1) : (1945, 12, 31)]
for item in period:
    war_records.append(item.text)    

In [154]:
with open('one_more_war_records.txt', 'w') as f:
    for item in war_records:
        f.write("%s\n" % item)

In [116]:
#dir(dw.notes[1])
#diary_number = re.search("@515", )
number = []
diary_number = re.compile("@"+"3\s")
records_number = re.search("\['\d{1,5}',\s'\d{1,5}',\s'.*?',\s'(?P<records>\d{1,4})'", str(line))
for line in dw.diaries:
    if re.search(diary_number, str(line)):
        number.append(len(line.notes))
number

[329]

In [327]:
#отделяем записи 20 века
twenty_cent = []
twenty = dw.notes[(1960, 1, 1) : (1970, 12, 31)]
#twenty_2 = dw.notes[(1948, 1, 1) : (1980, 12, 31)]
for item in twenty:
    twenty_cent.append(item.text) 

In [555]:
#ищем самых популярных авторов периода
authors5 = []
records = dw.notes[(1980, 1, 1) : (2017, 12, 31)]
for item in records:
    authors5.append(item.author)

In [556]:
authors5 = list(Counter(authors5).most_common())
authors_5 = pd.DataFrame(authors5)
authors_5.columns = ["Author", "1"]
#len(authors_1["Author"])

In [426]:
#ищем записи определенных авторов
amitrov_texts = []
kozakov_texts = []
shvets_texts = []
grobman_texts = []
sokolov_texts = []
records = dw.notes[(1948, 1, 1) : (1979, 12, 31)]
for item in records:
    author_kozakov = re.search(r"Николай Николаевич Козаков", str(item.author))
    author_amitrov = re.search(r"Олег Владимирович Амитров", str(item.author))
    author_shvets = re.search(r"Владимир Афанасьевич Швец", str(item.author))
    author_grobman = re.search(r"Михаил Яковлевич Гробман", str(item.author))
    author_sokolov = re.search(r"Николай Алексеевич Соколов", str(item.author))
    if author_kozakov:
        kozakov_texts.append(item.text)
    if author_amitrov:
        amitrov_texts.append(item.text)
    if author_shvets:
        shvets_texts.append(item.text)
    if author_grobman:
        grobman_texts.append(item.text)
    if author_sokolov:
        sokolov_texts.append(item.text)

In [339]:
amitrov_tokenized = []
kozakov_tokenized = []
for item in amitrov_texts:
    tokens = word_tokenize(str(item))
    for token in tokens:
        if token not in extended_punctuation:
            amitrov_tokenized.append(token)
for item in kozakov_texts:
    tokens = word_tokenize(str(item))
    for token in tokens:
        if token not in extended_punctuation:
            kozakov_tokenized.append(token)

In [353]:
shvets_tokenized = []
for item in shvets_texts:
    tokens = word_tokenize(str(item))
    for token in tokens:
        if token not in extended_punctuation:
            shvets_tokenized.append(token)
len(shvets_tokenized)

476548

In [341]:
len(amitrov_tokenized)

252230

In [342]:
len(kozakov_tokenized)

217389

In [352]:
with open('shvets_records.txt', 'w') as f:
    for item in shvets_texts:
        f.write("%s\n" % item)

In [338]:
#amitrov_texts
with open('amitrov_records.txt', 'w') as f:
    for item in amitrov_texts:
        f.write("%s\n" % item)
with open('kozakov_records.txt', 'w') as f:
    for item in kozakov_texts:
        f.write("%s\n" % item)

In [ ]:
#'1828-10-1') & (meta_info_cleaned['Дата'] <= '1898-12-31')]
#meta_2 = meta_info_cleaned[(meta_info_cleaned['Дата'] > '1898-12-31') & (meta_info_cleaned['Дата'] <= '1922-12-31')]
#meta_3 = meta_info_cleaned[(meta_info_cleaned['Дата'] > '1922-12-31') & (meta_info_cleaned['Дата'] <= '1938-12-31')]
#meta_4 = meta_info_cleaned[(meta_info_cleaned['Дата'] > '1938-12-31') & (meta_info_cleaned['Дата'] <= '1945-12-31')]
#meta_5 = meta_info_cleaned[(meta_info_cleaned['Дата'] > '1945-12-31') & (meta_info_cleaned['Дата'] <= '1959-12-31')]
#meta_6 = meta_info_cleaned[(meta_info_cleaned['Дата'] > '1959-12-31') & (meta_info_cleaned['Дата'] <= '1979-12-31')]
#meta_7 = meta_info_cleaned[(meta_info_cleaned['Дата'] > '1979-12-31') & (meta_info_cleaned['Дата'] <= '1999-12-31'

In [4]:
#считаем общее количество записей в периоде
period_1 = dw.notes[(1799, 1, 1) : (1898, 12, 31)]
period_2 = dw.notes[(1899, 1, 1) : (1938, 12, 31)]
period_3 = dw.notes[(1939, 1, 1) : (1947, 12, 31)]
period_4 = dw.notes[(1948, 1, 1) : (1979, 12, 31)]
period_5 = dw.notes[(1980, 1, 1) : (2017, 12, 31)]
#period_6 = dw.notes[(1960, 1, 1) : (1979, 12, 31)]
#period_7 = dw.notes[(1980, 1, 1) : (1998, 12, 31)]
#period_8 = dw.notes[(1999, 1, 1) : (2017, 12, 31)]

In [5]:
len(period_5)

53151

In [409]:
#считаем сколько записей в периоде без швеца
records_without_shvetsandko = []
for item in tqdm(period_4):
    author_shvets = re.search(r"Владимир Афанасьевич Швец", str(item.author))
    author_grobman = re.search(r"Михаил Яковлевич Гробман", str(item.author))
    author_sokolov = re.search(r"Николай Алексеевич Соколов", str(item.author))
    if author_shvets:
        continue
    elif author_grobman:
        continue
    elif author_sokolov:
        continue
    else:
        records_without_shvetsandko.append(item.text)
#len(records_without_shvets)

  0%|          | 0/73045 [00:00<?, ?it/s]

In [410]:
len(records_without_shvetsandko)

60714

In [329]:
#считаем количество токенов в периоде 
period4_tokenized = []
for item in twenty_cent:
    tokens = word_tokenize(str(item))
    for token in tokens:
        if token not in extended_punctuation:
            period4_tokenized.append(token)
#len(period1_tokenized)

In [330]:
len(period4_tokenized)

4304675

In [331]:
with open('twenty_cent_per42.txt', 'w') as f:
    for item in twenty_cent:
        f.write("%s\n" % item)

In [24]:
with open('old_records.txt', 'w') as f:
    for item in old_records:
        f.write("%s\n" % item)

In [202]:
#тестовая лемматизация
preprocess_text(dw.notes[1].text)

['ночью',
 'достигнуть',
 'банк',
 'антон-дорн',
 'датский',
 'пролив',
 'исландия',
 'гренландия',
 'утром',
 'погода',
 'препротивный',
 'сильный',
 'дождь',
 'сырость',
 'пронизывать',
 'ветер',
 'дночерпатель',
 'приносить',
 'чудесный',
 'коралл',
 'стащить',
 'вера',
 'владимирович',
 'подарить',
 'рождение',
 'вознаграждать',
 'неприятный',
 'труд',
 'трал',
 'принести',
 'хороший',
 'бентос',
 'основное',
 'губка',
 'масса',
 'больший',
 'фиолетовый',
 'медуза',
 'живой',
 'рыба',
 'полдень',
 'появиться',
 'айсберг',
 'фотографировать',
 'дождь',
 'кончиться',
 'спуститься',
 'густой',
 'туман',
 'судовой',
 'известие',
 'объявить',
 'гренландия',
 'лёд',
 'пускать',
 'далёкий',
 'прийтись',
 'идти',
 'север',
 'вдоль',
 'бо',
 'параллельно',
 'берег',
 'отправить',
 'поздравительный',
 'телеграмма',
 'сашка',
 'папа']

In [7]:
#собираем корпус с помощью регулярных выражений
diaries_dic = {}
info = {}
info_list = []
meta_info = pd.DataFrame(columns = ['ID записи', 'Автор', 'ID дневника', 'Дата'])
for i, diaries in enumerate(dw.notes):
    if re.search(r"я\s([а-яё-]+?\s)?влюб(ился|илась|лен(\s|а|ный|ная))", dw.notes[i].text.lower()):
    #if re.search(r"влюб(ленност.{1,2}|ился|илась|лен(\s|а|ный|ная))", dw.notes[i].text.lower()):
        diaries_dic[dw.notes[i].ID] = [dw.notes[i].text]
        try:
            info = {"ID" : dw.notes[i].ID, "Автор" : dw.notes[i].author, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        except KeyError:
            info = {"ID" : dw.notes[i].ID, "Автор" : 0, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        info_list.append(info) 

In [8]:
diaries_dic_again = {}
#meta_info = pd.DataFrame(columns = ['ID записи', 'Автор', 'ID дневника', 'Дата'])
# "Автор" : dw.notes[i].author
for i, diaries in enumerate(dw.notes):
    #if re.search(r"(он|она|я)\sизмени(л|ла)\s", dw.notes[i].text):
    if re.search(r"(я\s([а-яё-]+?\s)?разлюби(|л|ла))|(разлюби(л|ла|шь)\sменя)", dw.notes[i].text.lower()):
        diaries_dic[dw.notes[i].ID] = [dw.notes[i].text]
        try:
            info = {"ID" : dw.notes[i].ID, "Автор" : dw.notes[i].author, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        except KeyError:
            info = {"ID" : dw.notes[i].ID, "Автор" : 0, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        info_list.append(info)
for i, diaries in enumerate(dw.notes):
    if re.search(r"(он|она|я|ты)\sизмени(л|ла)\s(мне)?", dw.notes[i].text.lower()):
    #if re.search(r"разлюби(ть|л|ла)", dw.notes[i].text):
        diaries_dic[dw.notes[i].ID] = [dw.notes[i].text]
        try:
            info = {"ID" : dw.notes[i].ID, "Автор" : dw.notes[i].author, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        except KeyError:
            info = {"ID" : dw.notes[i].ID, "Автор" : 0, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        info_list.append(info)
for i, diaries in enumerate(dw.notes):
    if re.search(r"я\s([а-яё-]+?\s)?ревн(ую|овал|овала|ивый|ивая|ивец|ивица)", dw.notes[i].text.lower()):
    #if re.search(r"разлюби(ть|л|ла)", dw.notes[i].text):
        diaries_dic[dw.notes[i].ID] = [dw.notes[i].text]
        try:
            info = {"ID" : dw.notes[i].ID, "Автор" : dw.notes[i].author, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        except KeyError:
            info = {"ID" : dw.notes[i].ID, "Автор" : 0, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        info_list.append(info)

In [9]:
for i, diaries in enumerate(dw.notes):
    if re.search(r"ревн(овал|овала|уешь|ует)\sменя", dw.notes[i].text.lower()):
    #if re.search(r"разлюби(ть|л|ла)", dw.notes[i].text):
        diaries_dic[dw.notes[i].ID] = [dw.notes[i].text]
        try:
            info = {"ID" : dw.notes[i].ID, "Автор" : dw.notes[i].author, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        except KeyError:
            info = {"ID" : dw.notes[i].ID, "Автор" : 0, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        info_list.append(info)

In [10]:
for i, diaries in enumerate(dw.notes):
    if re.search(r"(любо?в.{0,2}\s(мо.{1,2}|наш.{1,2}|сво.{1,2}|его|её|ее|тво.{1,2}))|((мо.{1,2}|наш.{1,2}|сво.{1,2}|его|ее|её|тво.{1,2})\sлюбо?в.{0,2})", dw.notes[i].text.lower()):
        diaries_dic[dw.notes[i].ID] = [dw.notes[i].text]
        try:
            info = {"ID" : dw.notes[i].ID, "Автор" : dw.notes[i].author, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        except KeyError:
            info = {"ID" : dw.notes[i].ID, "Автор" : 0, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        info_list.append(info)
        meta_info = pd.DataFrame(info_list)  

In [11]:
 for i, diaries in enumerate(dw.notes):
    if re.search(r"любим.{0,3}\sм{2,4}|м{2,4}\sлюбим.{0,3}|я\sлюбим.{0,2}", dw.notes[i].text.lower()):
        diaries_dic[dw.notes[i].ID] = [dw.notes[i].text]
        try:
            info = {"ID" : dw.notes[i].ID, "Автор" : dw.notes[i].author, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        except KeyError:
            info = {"ID" : dw.notes[i].ID, "Автор" : 0, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        info_list.append(info)
        meta_info = pd.DataFrame(info_list)  

In [12]:
diaries_dic_again = {}
for i, diaries in enumerate(dw.notes):
    if re.search(r"я\s([а-яё-]+?\s)?(по)?люб(лю|я|ив|ил|ила|)\s(вас|ваш|тебя|твой|его|ее|её)", dw.notes[i].text.lower()):
        diaries_dic[dw.notes[i].ID] = [dw.notes[i].text]
        try:
            info = {"ID" : dw.notes[i].ID, "Автор" : dw.notes[i].author, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        except KeyError:
            info = {"ID" : dw.notes[i].ID, "Автор" : 0, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        info_list.append(info)
        meta_info = pd.DataFrame(info_list) 

In [13]:
diaries_dic_again = {}
for i, diaries in enumerate(dw.notes):
    if re.search(r"(ты|он|она)\s(по|не\s)?люб(ишь|я|ив|ил|ила|ит)\sменя", dw.notes[i].text.lower()):
        diaries_dic[dw.notes[i].ID] = [dw.notes[i].text]
        try:
            info = {"ID" : dw.notes[i].ID, "Автор" : dw.notes[i].author, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        except KeyError:
            info = {"ID" : dw.notes[i].ID, "Автор" : 0, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        info_list.append(info)
        meta_info = pd.DataFrame(info_list)

In [14]:
for i, diaries in enumerate(dw.notes):
    if re.search(r"я?\s([а-яё-]+?\s)?(вас|тебя|меня)\s([а-яё,\s-]+?)?(по)?люб(лю|я|ив|ил|ила|ишь)", dw.notes[i].text.lower()):
        diaries_dic[dw.notes[i].ID] = [dw.notes[i].text]
        try:
            info = {"ID" : dw.notes[i].ID, "Автор" : dw.notes[i].author, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        except KeyError:
            info = {"ID" : dw.notes[i].ID, "Автор" : 0, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        info_list.append(info)
        meta_info = pd.DataFrame(info_list) 

In [15]:
diaries_dic_again = {}
diaries_dic_more = {}
for i, diaries in enumerate(dw.notes):
    #if re.search(r"поцел(овал|овала|уй)\sменя", dw.notes[i].text.lower()):
    if re.search(r"перв.{1,3}поцелу.{1,3}", dw.notes[i].text.lower()):
        diaries_dic[dw.notes[i].ID] = [dw.notes[i].text]
    #if re.search(r"настоящ.{2,3}\sлюбо?в{1,2}", dw.notes[i].text.lower()):
    #    diaries_dic_more[dw.notes[i].ID] = [dw.notes[i].text]
        try:
            info = {"ID" : dw.notes[i].ID, "Автор" : dw.notes[i].author, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        except KeyError:
            info = {"ID" : dw.notes[i].ID, "Автор" : 0, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        info_list.append(info)
        meta_info = pd.DataFrame(info_list) 

In [16]:
diaries_dic_again = {}
for i, diaries in enumerate(dw.notes):
    if re.search(r"(призна(лся|лась)\s(мне\sв\sлюбви|в\sлюбви\sмне))|(признани.{1,2}\sв\sлюбви)", dw.notes[i].text.lower()):
        diaries_dic[dw.notes[i].ID] = [dw.notes[i].text]
        try:
            info = {"ID" : dw.notes[i].ID, "Автор" : dw.notes[i].author, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        except KeyError:
            info = {"ID" : dw.notes[i].ID, "Автор" : 0, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        info_list.append(info)
        meta_info = pd.DataFrame(info_list) 

In [17]:
for i, diaries in enumerate(dw.notes):
    if re.search(r"(мо.{1,3}\sволокитств.{1,2})|(волочи(лся|ться)\sза\sмной)|(я\s([а-яё-]+?\s)?волочи(лся|ться))", dw.notes[i].text.lower()):
        diaries_dic[dw.notes[i].ID] = [dw.notes[i].text]
        try:
            info = {"ID" : dw.notes[i].ID, "Автор" : dw.notes[i].author, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        except KeyError:
            info = {"ID" : dw.notes[i].ID, "Автор" : 0, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        info_list.append(info)
        meta_info = pd.DataFrame(info_list) 

In [18]:
diaries_dic_again = {}
diaries_dic_more = {}
for i, diaries in enumerate(dw.notes):
    if re.search(r"любим.{2,3}\s([а-яё-]+?\s)?человек.{0,2}", dw.notes[i].text.lower()):
        diaries_dic[dw.notes[i].ID] = [dw.notes[i].text]
        try:
            info = {"ID" : dw.notes[i].ID, "Автор" : dw.notes[i].author, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        except KeyError:
            info = {"ID" : dw.notes[i].ID, "Автор" : 0, "ID дневника" : dw.notes[i].diary_ID, "Дата" : dw.notes[i].date}
        info_list.append(info)
        meta_info = pd.DataFrame(info_list) 
    #if re.search(r"настоящ.{2,3}\sлюбо?в{1,2}", dw.notes[i].text.lower()):
    #    diaries_dic_more[dw.notes[i].ID] = [dw.notes[i].text]

In [19]:
len(diaries_dic)

6283

In [254]:
#сколько всего записей в корпусе
#len(diaries_dic)
#len(diaries_dic)

In [154]:
#отбираем записи 20 века
twenty_cent = []
for item in diaries_dic.items():
    if re.search("\(19\d{2},\s\d{1,2},\s\d{1,2}\)", str(item[0])):
    #if key.startswith('(1910'):
        twenty_cent.append(item)
#[key for key, value in diaries_dic if key.startswith('(1910')]

In [156]:
#twenty_cent

In [161]:
#так тоже можно выбирать записи 
regex = '\((17|18).*\)'

results = [v for k, v in diaries_dic.items() if re.match(regex, str(k))]

In [254]:
#results

In [255]:
#ищем повторяющиеся записи в метаданных
pd.concat(g for _, g in meta_info.groupby("ID") if len(g) > 1)

,ID,Автор,ID дневника,Дата
1480,1471,0,2,"(1917, 2, 14)"
4291,1471,0,2,"(1917, 2, 14)"
84,2404,#13 Александр Александрович Блок,10,"(1911, 12, 1)"
5652,2404,#13 Александр Александрович Блок,10,"(1911, 12, 1)"
118,2869,#13 Александр Александрович Блок,10,"(1920, 11, 7)"
...,...,...,...,...
6947,467461,#1478 Нелли,1488,"(1917, 9, 25)"
1725,468335,#3334 Ольга Ивановна Баранова,3344,"(1923, 1, 21)"
5835,468335,#3334 Ольга Ивановна Баранова,3344,"(1923, 1, 21)"
199,468933,#3477 Эра Гансовна Мундецем,3487,"(1944, 6, 8)"


In [32]:
#убираем их
meta_info_cleaned = meta_info.drop_duplicates(subset=['ID'])
meta_info_cleaned

,ID,Автор,ID дневника,Дата
0,140292,#507 Степан Петрович Жихарев,515,"(1805, 7, 20)"
1,444870,#719 Николай Дмитриевич Дурново,729,"(1812, 1, 22)"
2,178659,#763 Анна Алексеевна Андро,773,"(1828, 10, 6)"
3,171755,#706 Алексей Николаевич Вульф,716,"(1831, 2, 10)"
4,442455,#790 Дарья Фёдоровна Фикельмон,800,"(1831, 7, 5)"
...,...,...,...,...
7136,435616,#1949 Юрий Михайлович Кублановский,1959,"(2010, 3, 14)"
7137,394573,#2338 Ирина Львовна Шолохова,2348,"(2011, 1, 3)"
7139,394581,#2338 Ирина Львовна Шолохова,2348,"(2012, 12, 2)"
7140,394582,#2338 Ирина Львовна Шолохова,2348,"(2012, 12, 29)"


In [40]:
#превращаем дату в datetime object
#meta_info_cleaned.info()
#meta_info_cleaned['Автор']= meta_info_cleaned['Автор'].astype(str)
#meta_info_cleaned['Дата']= meta_info_cleaned['Дата'].astype('datetime64[ns]')
meta_info_cleaned['Дата'] = pd.to_datetime(meta_info_cleaned['Дата'], format='(%Y, %m, %d)')
meta_info_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6283 entries, 0 to 7142
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   ID           6283 non-null   int64         
 1   Автор        6280 non-null   object        
 2   ID дневника  6283 non-null   int64         
 3   Дата         6283 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 245.4+ KB


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [298]:
#отбираем записи определенного периода
#meta_info_cleaned[meta_info_cleaned.index.isin(['2010-1-2', '2017-6-8'])]
#print(meta_info_cleaned.loc['2000-1-1':'2017-6-8'])
#meta_1 = meta_info_cleaned[(meta_info_cleaned['Дата'] > '1828-10-1') & (meta_info_cleaned['Дата'] <= '1898-12-31')]
#meta_2 = meta_info_cleaned[(meta_info_cleaned['Дата'] > '1898-12-31') & (meta_info_cleaned['Дата'] <= '1922-12-31')]
#meta_3 = meta_info_cleaned[(meta_info_cleaned['Дата'] > '1922-12-31') & (meta_info_cleaned['Дата'] <= '1938-12-31')]
#meta_4 = meta_info_cleaned[(meta_info_cleaned['Дата'] > '1938-12-31') & (meta_info_cleaned['Дата'] <= '1945-12-31')]
#meta_5 = meta_info_cleaned[(meta_info_cleaned['Дата'] > '1945-12-31') & (meta_info_cleaned['Дата'] <= '1959-12-31')]
#meta_6 = meta_info_cleaned[(meta_info_cleaned['Дата'] > '1959-12-31') & (meta_info_cleaned['Дата'] <= '1979-12-31')]
#meta_7 = meta_info_cleaned[(meta_info_cleaned['Дата'] > '1979-12-31') & (meta_info_cleaned['Дата'] <= '1998-12-31')]
#meta_8 = meta_info_cleaned[(meta_info_cleaned['Дата'] > '1998-12-31') & (meta_info_cleaned['Дата'] <= '2017-12-31')]

In [86]:
#meta_1 771 запись -- 46107 записей всего
#meta_2 1437 записей -- 79599 записей всего
#meta_3 805 записей -- 50995 записей всего
#meta_4 1186 записей -- 68083 записей всего
#meta_5 468 записей -- 31066 записей всего
#meta_6 635 записей -- 47007 записей всего
#meta_7 747 записи -- 40143 записей всего
#meta_8 356 записей -- 13008 записей всего

In [42]:
#отбираем записи определенного периода
meta_1 = meta_info_cleaned[(meta_info_cleaned['Дата'] > '1798-12-31') & (meta_info_cleaned['Дата'] <= '1898-12-31')]
meta_2 = meta_info_cleaned[(meta_info_cleaned['Дата'] > '1898-12-31') & (meta_info_cleaned['Дата'] <= '1938-12-31')]
meta_3 = meta_info_cleaned[(meta_info_cleaned['Дата'] > '1938-12-31') & (meta_info_cleaned['Дата'] <= '1947-12-31')]
meta_4 = meta_info_cleaned[(meta_info_cleaned['Дата'] > '1947-12-31') & (meta_info_cleaned['Дата'] <= '1979-12-31')]
meta_5 = meta_info_cleaned[(meta_info_cleaned['Дата'] > '1979-12-31') & (meta_info_cleaned['Дата'] <= '2017-12-31')]

In [622]:
#len(meta_1) #818 записи -- 50467 записей всего 0.016
#len(meta_2) #2207 записи -- 130594 записей всего 0.017
#len(meta_3) #1301 записей -- 73111 записей всего 0.018
#len(meta_4) #936 записи -- 73045 записей всего 0.013
#len(meta_5) #1017 записи -- 53151 записей всего 0.019

1017

In [125]:
#отбираем ID записей определенного периода
period1_ids = []
period2_ids = []
period3_ids = []
period4_ids = []
period5_ids = []
for item in meta_1["ID"]:
    period1_ids.append(item)
for item in meta_2["ID"]:
    period2_ids.append(item)
for item in meta_3["ID"]:
    period3_ids.append(item)
for item in meta_4["ID"]:
    period4_ids.append(item)
for item in meta_5["ID"]:
    period5_ids.append(item)

In [619]:
#собираем записи разных периодов в списки (способ нерабочий, удалять жалко)
#period1_records = []
#period1_ids_regex = re.compile(""+"|".join(str(x) for x in period1_ids))
#for item in diaries_dic.items():
#    period1_match = re.search(period1_ids_regex, str(item[0]))
#    if period1_match:
#        period1_records.append(item[1])

In [126]:
#собираем записи разных периодов в списки
period1_records = []
period2_records = []
period3_records = []
period4_records = []
period5_records = []
for item in diaries_dic.items():
    for x in period1_ids:
        if item[0] == x:
            period1_records.append(item[1])
    for x in period2_ids:
        if item[0] == x:
            period2_records.append(item[1])
    for x in period3_ids:
        if item[0] == x:
            period3_records.append(item[1])
    for x in period4_ids:
        if item[0] == x:
            period4_records.append(item[1])
    for x in period5_ids:
        if item[0] == x:
            period5_records.append(item[1])

In [127]:
len(period4_records)

936

In [169]:
len(period1_records)

818

In [167]:
#считаем количество токенов в периоде
period5_tokenized = []
for item in period5_records:
    tokens = word_tokenize(str(item))
    for token in tokens:
        if token not in extended_punctuation:
            period5_tokenized.append(token)
#len(period1_tokenized)

In [168]:
len(period5_tokenized)

458551

In [147]:
#period1_tokenized

In [708]:
#выгружаем записи в txt
with open('period1_records.txt', 'w') as p1:
    for item in period1_records:
        p1.write("%s\n" % item)
with open('period3_records.txt', 'w') as p3:
    for item in period3_records:
        p3.write("%s\n" % item)
with open('period4_records.txt', 'w') as p4:
    for item in period4_records:
        p4.write("%s\n" % item)

In [108]:
with open('period2_records.txt', 'w') as p2:
    for item in period2_records:
        p2.write("%s\n" % item)
with open('period5_records.txt', 'w') as p5:
    for item in period5_records:
        p5.write("%s\n" % item)

In [317]:
#meta_info_cleaned[(meta_info_cleaned['Дата'] > '2013-1-1') & (meta_info_cleaned['Дата'] < '2017-6-8')]
#meta_info_cleaned[(meta_info_cleaned['Дата']>datetime.date(2016, 1, 1)) & (meta_info_cleaned['Дата']<datetime.date(2017, 6, 8))]

In [37]:
#отбираем имена, чтобы разделить по гендеру
names = set()
for item in meta_info_cleaned["Автор"]:
    name = re.search("#\d{1,5}\s(?P<name>[А-ЯЁ][а-яё]+)", str(item))
    if name:
        names.add(name.group ("name"))

In [38]:
female = []
male = []
ambig_fem = ["Любовь", "Рашель", "Эсфирь", "Элен", "Нуне"]
ambig_male = ["Микола", "Илья", "Дзига", "Никита"] 
for name in names:
    if re.search("(а|я|и)$", name) and name not in ambig_male:
        female.append(name)
    elif name in ambig_fem:
        female.append(name)
    else:
         male.append(name)

In [39]:
female_regex = re.compile(""+"|".join(female))
#regex = re.compile("^("+"|".join(item_list)+")\.")
male_regex = re.compile(""+"|".join(male))

In [40]:
#создаем новый столбец с гендером
gender = []
for item in meta_info_cleaned["Автор"]:
    fem_match = re.search(female_regex, str(item))
    male_match = re.search(male_regex, str(item))
    if fem_match:
        gender.append("Женский")
    elif male_match:
        gender.append("Мужской")
    else:
        gender.append("")
meta_info_cleaned["Гендер"] = gender

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [41]:
meta_info_cleaned

,ID,Автор,ID дневника,Дата,Гендер
0,140292,#507 Степан Петрович Жихарев,515,1805-07-20,Мужской
1,444870,#719 Николай Дмитриевич Дурново,729,1812-01-22,Мужской
2,178659,#763 Анна Алексеевна Андро,773,1828-10-06,Женский
3,171755,#706 Алексей Николаевич Вульф,716,1831-02-10,Мужской
4,442455,#790 Дарья Фёдоровна Фикельмон,800,1831-07-05,Женский
...,...,...,...,...,...
7136,435616,#1949 Юрий Михайлович Кублановский,1959,2010-03-14,Мужской
7137,394573,#2338 Ирина Львовна Шолохова,2348,2011-01-03,Женский
7139,394581,#2338 Ирина Львовна Шолохова,2348,2012-12-02,Женский
7140,394582,#2338 Ирина Львовна Шолохова,2348,2012-12-29,Женский


In [42]:
#приводим метаданные к единому формату
ids = []
for item in meta_info_cleaned["Автор"]:
    number = re.search(r"\d{1,5}", str(item))
    if number:
        ids.append(number.group())
    else: 
        ids.append(0)
        
p = re.compile(r"#\d{1,5}\s")
meta_info_cleaned["Автор"] = [p.sub('', str(x)) for x in meta_info_cleaned["Автор"]]
        
meta_info_cleaned["ID автора"] = ids   
meta_info_cleaned

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,ID,Автор,ID дневника,Дата,Гендер,ID автора
0,140292,Степан Петрович Жихарев,515,1805-07-20,Мужской,507
1,444870,Николай Дмитриевич Дурново,729,1812-01-22,Мужской,719
2,178659,Анна Алексеевна Андро,773,1828-10-06,Женский,763
3,171755,Алексей Николаевич Вульф,716,1831-02-10,Мужской,706
4,442455,Дарья Фёдоровна Фикельмон,800,1831-07-05,Женский,790
...,...,...,...,...,...,...
7136,435616,Юрий Михайлович Кублановский,1959,2010-03-14,Мужской,1949
7137,394573,Ирина Львовна Шолохова,2348,2011-01-03,Женский,2338
7139,394581,Ирина Львовна Шолохова,2348,2012-12-02,Женский,2338
7140,394582,Ирина Львовна Шолохова,2348,2012-12-29,Женский,2338


In [43]:
meta_info_cleaned['ID автора']= meta_info_cleaned['ID автора'].astype(int)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [44]:
#функция для колонки с количеством записей в дневнике
def record_number(row):
    #for item in tqdm(row["ID дневника"]):
    diary_number = re.compile("@"+str(row["ID дневника"])+"\s")
    for line in dw.diaries:
        if re.search(diary_number, str(line)):
            number = len(line.notes)
            return number
meta_info_cleaned['Количество записей'] = meta_info_cleaned.apply(record_number, axis=1)

KeyboardInterrupt: 

In [36]:
#создаём колонку с периодом записи
period_column = []
for item in tqdm(meta_info_cleaned['Дата']):
    #item = datetime.strptime(item, "%Y-%m-%d")
    if item > datetime.strptime('1798-12-31', "%Y-%m-%d") and item <= datetime.strptime('1898-12-31', "%Y-%m-%d"):
        period_column.append("Период 1")
    elif item > datetime.strptime('1898-12-31', "%Y-%m-%d") and item <= datetime.strptime('1938-12-31', "%Y-%m-%d"):
        period_column.append("Период 2")
    elif item > datetime.strptime('1938-12-31', "%Y-%m-%d") and item <= datetime.strptime('1947-12-31', "%Y-%m-%d"):
        period_column.append("Период 3")
    elif item > datetime.strptime('1947-12-31', "%Y-%m-%d") and item <= datetime.strptime('1979-12-31', "%Y-%m-%d"):
        period_column.append("Период 4")
    elif item > datetime.strptime('1979-12-31', "%Y-%m-%d") and item <= datetime.strptime('2017-12-31', "%Y-%m-%d"):
        period_column.append("Период 5")
    else:
        period_column.append("")

  0%|          | 0/6283 [00:00<?, ?it/s]

In [37]:
#meta_info_cleaned["Количество записей"] = numbers 
meta_info_cleaned["Период"] = period_column

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [38]:
meta_info_cleaned

,ID,Автор,ID дневника,Дата,Гендер,ID автора,Период
0,140292,Степан Петрович Жихарев,515,1805-07-20,Мужской,507,Период 1
1,444870,Николай Дмитриевич Дурново,729,1812-01-22,Мужской,719,Период 1
2,178659,Анна Алексеевна Андро,773,1828-10-06,Женский,763,Период 1
3,171755,Алексей Николаевич Вульф,716,1831-02-10,Мужской,706,Период 1
4,442455,Дарья Фёдоровна Фикельмон,800,1831-07-05,Женский,790,Период 1
...,...,...,...,...,...,...,...
7136,435616,Юрий Михайлович Кублановский,1959,2010-03-14,Мужской,1949,Период 5
7137,394573,Ирина Львовна Шолохова,2348,2011-01-03,Женский,2338,Период 5
7139,394581,Ирина Львовна Шолохова,2348,2012-12-02,Женский,2338,Период 5
7140,394582,Ирина Львовна Шолохова,2348,2012-12-29,Женский,2338,Период 5


In [148]:
meta_info_cleaned.to_csv("prozhito_meta.csv", index=False)

In [40]:
#объединяем датафреймы
full_meta_data = pd.merge(meta_info_cleaned, authors_data, left_on='ID автора', right_on='id', how='left').drop(['id', 'first_name', 'family_name', 'patronymic', 'gender', 'from_natasha', 'from_tags'], axis=1)

In [41]:
full_meta_data

,ID,Автор,ID дневника,Дата,Гендер,ID автора,Период,birth_date,death_date,edition,additional_info,info,nickname,wiki
0,140292,Степан Петрович Жихарев,515,1805-07-20,Мужской,507,Период 1,1787-02-28,1860-09-12,**Издание:** [* Жихарев С.П.* Записки современ...,NaN,"Писатель, драматург, переводчик, сенатор, теат...",NaN,https://ru.wikipedia.org/wiki/%D0%96%D0%B8%D1%...
1,444870,Николай Дмитриевич Дурново,729,1812-01-22,Мужской,719,Период 1,1725-01-01,1816-12-31,**Издание:** [1812 год... Военные дневники / С...,NaN,"Генерал-аншеф, сенатор.",NaN,"https://ru.wikipedia.org/wiki/Дурново,_Николай..."
2,178659,Анна Алексеевна Андро,773,1828-10-06,Женский,763,Период 1,1808-08-23,1888-12-27,**Издание:** \n* *Оленина А.А. *Дневник. Воспо...,**Критика комментариев к изданию 1999 года: **...,Возлюбленная А.С. Пушкина.,Оленина,"https://ru.wikipedia.org/wiki/Оленина,_Анна_Ал..."
3,171755,Алексей Николаевич Вульф,716,1831-02-10,Мужской,706,Период 1,1805-12-28,1881-04-29,**Издания:** \n* *А. Н. Вульф*. Дневники : Люб...,NaN,Мемуарист. Близкий друг [А. С. Пушкина](http:/...,NaN,"https://ru.wikipedia.org/wiki/Вульф,_Алексей_Н..."
4,442455,Дарья Фёдоровна Фикельмон,800,1831-07-05,Женский,790,Период 1,1804-10-26,1863-04-22,**Издания:**\n\n* *Kauchtschwili N.* Il diario...,* *Гиллельсон М. И.* Пушкин в итальянском изда...,Внучка [М. И. Кутузова](http://prozhito.org/pe...,Долли,"https://ru.wikipedia.org/wiki/Фикельмон,_Дарья..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6278,435616,Юрий Михайлович Кублановский,1959,2010-03-14,Мужской,1949,Период 5,1947-04-30,NaN,**Издание:** *Кублановский Ю.* Год за год // Н...,NaN,"Поэт и эссеист, публицист, критик, искусствовед.",NaN,"https://ru.wikipedia.org/wiki/Кублановский,_Юр..."
6279,394573,Ирина Львовна Шолохова,2348,2011-01-03,Женский,2338,Период 5,1971-04-15,NaN,Мы сердечно благодарим [автора](https://vk.com...,NaN,Ведущий специалист литературной части Санкт-Пе...,NaN,NaN
6280,394581,Ирина Львовна Шолохова,2348,2012-12-02,Женский,2338,Период 5,1971-04-15,NaN,Мы сердечно благодарим [автора](https://vk.com...,NaN,Ведущий специалист литературной части Санкт-Пе...,NaN,NaN
6281,394582,Ирина Львовна Шолохова,2348,2012-12-29,Женский,2338,Период 5,1971-04-15,NaN,Мы сердечно благодарим [автора](https://vk.com...,NaN,Ведущий специалист литературной части Санкт-Пе...,NaN,NaN


In [42]:
full_meta_data['birth_date'] = pd.to_datetime(full_meta_data['birth_date'], format='%Y-%m-%d')
full_meta_data['death_date'] = pd.to_datetime(full_meta_data['death_date'], format='%Y-%m-%d', errors = 'coerce')
full_meta_data['Дата'] = pd.to_datetime(full_meta_data['Дата'], format='(%Y, %m, %d)')

In [43]:
full_meta_data['date_year'] = full_meta_data.Дата.apply(lambda x: x.year)
full_meta_data['birth_date_year'] = full_meta_data.birth_date.apply(lambda x: x.year)
full_meta_data['age'] = full_meta_data['date_year'] - full_meta_data['birth_date_year']

In [44]:
full_meta_data.drop(['date_year', 'birth_date_year'], axis=1)

,ID,Автор,ID дневника,Дата,Гендер,ID автора,Период,birth_date,death_date,edition,additional_info,info,nickname,wiki,age
0,140292,Степан Петрович Жихарев,515,1805-07-20,Мужской,507,Период 1,1787-02-28,1860-09-12,**Издание:** [* Жихарев С.П.* Записки современ...,NaN,"Писатель, драматург, переводчик, сенатор, теат...",NaN,https://ru.wikipedia.org/wiki/%D0%96%D0%B8%D1%...,18.0
1,444870,Николай Дмитриевич Дурново,729,1812-01-22,Мужской,719,Период 1,1725-01-01,1816-12-31,**Издание:** [1812 год... Военные дневники / С...,NaN,"Генерал-аншеф, сенатор.",NaN,"https://ru.wikipedia.org/wiki/Дурново,_Николай...",87.0
2,178659,Анна Алексеевна Андро,773,1828-10-06,Женский,763,Период 1,1808-08-23,1888-12-27,**Издание:** \n* *Оленина А.А. *Дневник. Воспо...,**Критика комментариев к изданию 1999 года: **...,Возлюбленная А.С. Пушкина.,Оленина,"https://ru.wikipedia.org/wiki/Оленина,_Анна_Ал...",20.0
3,171755,Алексей Николаевич Вульф,716,1831-02-10,Мужской,706,Период 1,1805-12-28,1881-04-29,**Издания:** \n* *А. Н. Вульф*. Дневники : Люб...,NaN,Мемуарист. Близкий друг [А. С. Пушкина](http:/...,NaN,"https://ru.wikipedia.org/wiki/Вульф,_Алексей_Н...",26.0
4,442455,Дарья Фёдоровна Фикельмон,800,1831-07-05,Женский,790,Период 1,1804-10-26,1863-04-22,**Издания:**\n\n* *Kauchtschwili N.* Il diario...,* *Гиллельсон М. И.* Пушкин в итальянском изда...,Внучка [М. И. Кутузова](http://prozhito.org/pe...,Долли,"https://ru.wikipedia.org/wiki/Фикельмон,_Дарья...",27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6278,435616,Юрий Михайлович Кублановский,1959,2010-03-14,Мужской,1949,Период 5,1947-04-30,NaT,**Издание:** *Кублановский Ю.* Год за год // Н...,NaN,"Поэт и эссеист, публицист, критик, искусствовед.",NaN,"https://ru.wikipedia.org/wiki/Кублановский,_Юр...",63.0
6279,394573,Ирина Львовна Шолохова,2348,2011-01-03,Женский,2338,Период 5,1971-04-15,NaT,Мы сердечно благодарим [автора](https://vk.com...,NaN,Ведущий специалист литературной части Санкт-Пе...,NaN,NaN,40.0
6280,394581,Ирина Львовна Шолохова,2348,2012-12-02,Женский,2338,Период 5,1971-04-15,NaT,Мы сердечно благодарим [автора](https://vk.com...,NaN,Ведущий специалист литературной части Санкт-Пе...,NaN,NaN,41.0
6281,394582,Ирина Львовна Шолохова,2348,2012-12-29,Женский,2338,Период 5,1971-04-15,NaT,Мы сердечно благодарим [автора](https://vk.com...,NaN,Ведущий специалист литературной части Санкт-Пе...,NaN,NaN,41.0


In [44]:
#считаем возрастные группы
counter1 = 0
counter2 = 0
counter3 = 0
counter4 = 0
counter5 = 0
for item in full_meta_data["age"]:
    if item <= 16:
        counter1 += 1
for item in full_meta_data["age"]:
    if item > 16 and item <= 26: 
        counter2 += 1
for item in full_meta_data["age"]:
    if item > 26 and item <= 41: 
        counter3 += 1
for item in full_meta_data["age"]:
    if item > 41 and item <= 60: 
        counter4 += 1
for item in full_meta_data["age"]:
    if item > 60: 
        counter5 += 1
print(counter1, counter2, counter3, counter4, counter5)

440 1705 1486 1283 894


In [407]:
#meta_info_cleaned[meta_info_cleaned['Автор'].str.contains("#78 Евгений Львович Шварц")]

In [45]:
#Объединяем авторов в один датафрейм, чтобы посчитать среднее эмоциональное

authors4_corpus = meta_4.Автор.value_counts()
authors4_corpus = pd.DataFrame(authors4_corpus)
authors4_corpus = authors4_corpus.reset_index()
authors4_corpus.columns = ["Author", "2"]

In [47]:
len(authors4_corpus["Author"])

138

In [48]:
authors4_full = pd.merge(authors_4, authors4_corpus, on="Author")

NameError: name 'authors_4' is not defined

In [494]:
new_column = authors4_full["2"] / authors4_full["1"]
authors4_full["3"] = new_column
authors4_full["3"].mean()

0.054407282287850876

In [500]:
#То же самое с 3 периодом
authors3_corpus = meta_3.Автор.value_counts()
authors3_corpus = pd.DataFrame(authors3_corpus)
authors3_corpus = authors3_corpus.reset_index()
authors3_corpus.columns = ["Author", "2"]

In [501]:
authors3_full = pd.merge(authors_3, authors3_corpus, on="Author")

In [502]:
new_column = authors3_full["2"] / authors3_full["1"]
authors3_full["3"] = new_column
authors3_full["3"].mean()

0.05215995940294942

In [512]:
#Второй период
authors2_corpus = meta_2.Автор.value_counts()
authors2_corpus = pd.DataFrame(authors2_corpus)
authors2_corpus = authors2_corpus.reset_index()
authors2_corpus.columns = ["Author", "2"]

In [513]:
authors2_full = pd.merge(authors_2, authors2_corpus, on="Author")
new_column = authors2_full["2"] / authors2_full["1"]
authors3_full["3"] = new_column
authors3_full["3"].mean()

0.042479003633874815

In [514]:
#Первый период
authors1_corpus = meta_1.Автор.value_counts()
authors1_corpus = pd.DataFrame(authors1_corpus)
authors1_corpus = authors1_corpus.reset_index()
authors1_corpus.columns = ["Author", "2"]

In [515]:
authors1_full = pd.merge(authors_1, authors1_corpus, on="Author")
new_column = authors1_full["2"] / authors1_full["1"]
authors1_full["3"] = new_column
authors1_full["3"].mean()

0.06629841037155669

In [527]:
#Пятый период
authors5_corpus = meta_5.Автор.value_counts()
authors5_corpus = pd.DataFrame(authors5_corpus)
authors5_corpus = authors5_corpus.reset_index()
authors5_corpus.columns = ["Author", "2"]

In [528]:
authors5_full = pd.merge(authors_5, authors5_corpus, on="Author")
new_column = authors5_full["2"] / authors5_full["1"]
authors5_full["3"] = new_column
authors5_full["3"].mean()

0.04132390911505409

In [47]:
#meta_4["Автор"].count()
#pd.set_option("display.max_rows", 20, "display.max_columns", None)
meta_5.Автор.value_counts()

#981 Галина Г. Ларская                  280
#491 Елена Викторовна Полюшкина          93
#102 Валерий Сергеевич Золотухин         69
#878 Сергей Николаевич Есин              56
#621 Владимир Александрович Бессонов     35
                                       ... 
#1794 Семён Борисович Ласкин              1
#1309 Игорь  Холодяков                    1
#1248 Катарина  Венцль                    1
#531 Лев Абрамович Остерман               1
#3381 Руслан Геннадьевич Сидоров          1
Name: Автор, Length: 98, dtype: int64

In [91]:
#считаем, сколько женщин и мужчин в корпусе
#meta_info_cleaned.groupby('Автор')('Гендер').value_counts()
meta_info_cleaned.groupby('Гендер')['Автор'].nunique()

Гендер
            12
Женский    243
Мужской    444
Name: Автор, dtype: int64

In [53]:
#выбираем ID записей женщин и мужчин
female_ids = []
male_ids = []
for item in meta_info_cleaned[meta_info_cleaned['Гендер'].str.match('Женский')]["ID"]:
    female_ids.append(item)
for item in meta_info_cleaned[meta_info_cleaned['Гендер'].str.match('Мужской')]["ID"]:
    male_ids.append(item)

KeyError: 'Гендер'

In [641]:
#len(female_ids)

2618

In [655]:
#собираем записи женщин и мужчин в списки
female_records = []
male_records = []
for item in diaries_dic.items():
    for x in female_ids:
        if item[0] == x:
            female_records.append(item[1])
       #female_records.append(item)
    for x in male_ids:
        if item[0] == x:
            male_records.append(item[1])      

In [656]:
len(female_records)

2618

In [84]:
#выгружаем записи в txt
with open('female_records.txt', 'w') as f:
    for item in female_records:
        f.write("%s\n" % item)

In [85]:
with open('male_records.txt', 'w') as m:
    for item in male_records:
        m.write("%s\n" % item)

In [390]:
#не получилось
fem = []
for item in diaries_dic.items():
    if item[0] == (x for x in female_ids):
        fem.append(item)

In [188]:
#лемматизируем корпус
for key, value in diaries_dic.items():
    diaries_dic[key] = preprocess_text(diaries_dic[key])

In [280]:
#diaries_dic

In [25]:
def flatten_list(_2d_list):
    '''
    берет вложенный список, возвращает плоский
    '''
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

In [42]:
period1_records_flat = flatten_list(period1_records)

In [455]:
#экспериментальный чанк 
period1_tokenized_records = []
for item in period1_records_flat:
    items = word_tokenize(item.lower())
    period1_tokenized_records.append(items)

In [46]:
all_records = []
for item in diaries_dic.items():
    all_records.append(item[1])

In [47]:
all_records_flat = flatten_list(all_records)

In [48]:
#period1_records_2
periods_lemmatized = []
for item in all_records_flat:
    periods_lemmatized.append(preprocess_text(item))

In [43]:
period1_lemmatized = []
for item in period1_records_flat:
    period1_lemmatized.append(preprocess_text(item))

In [54]:
names_list = ["степанович", "алик", "люся", "ир", "екатерина", "лёня", "люба", "герман", "митя", "давид", "вячеслав", "алла", "галя", "эфроса", "витя", "губенко", "лина", "наталья", "мишка", "валерий", "дима", "лариса", "аня", "ляла", "татьяна", "аксюша", "юзеф", "дмитрий", "алексей", "дмитриевич", "петров", "петя", "андрей", "гриша", "боря", "николаев", "аля", "елена", "сереж", "григориевич", "василий", "сомовый", "павлик", "шура", "наташа", "катя", "александрович", "иван", "юрий", "надя", "владимир", "лида", "виктор", "иванов", "ирина", "валя", "борис", "анович", "мария", "чертков", "сергеевич", "ольга", "лёля", "фёдорович", "григорий", "михаилович", "николаевич", "анна", "пётр", "петрович", "васильевич", "ляля", "иванович", "зина", "володя", "никита", "лена", "коля", "николай", "маруся", "лиза", "олег", "вася", "игорь", "георгий", "львович", "марина", "софья", "андреевич", "александр", "цветаев", "владимирович", "таня", "анна", "василиевич", "петрович", "сергей", "василиевич", "саша", "павлович", "маша", "миша", "константин", "пётр", "павел", "лена", "лев", "серёжа", "оля", "яковлевич", "нина", "шурик", "женя", "соня", "семён", "костя", "миша"]
for item in periods_lemmatized:
    for i in range(len(item)):
        if item[i] in names_list:
            item[i] = "<имя>"
        if re.search("[а-я]\.", str(item[i])):
            item[i] = "<имя>"
    #item = [element for element in item if re.search("^(?!(?:[а-я]\.)$).*$", str(element))]
    #if re.search("[а-я]\.", records_lemmatized[i])

#period1_lemmatized = [item for item in period1_lemmatized if re.search("^(?!(?:[а-я]\.)$).*$", str(item))]

In [659]:
#period1_lemmatized_flat = flatten_list(period1_lemmatized)

In [255]:
#period1_corrected

In [44]:
period1_corrected = [" ".join(x) for x in period1_lemmatized]

In [49]:
def get_topics(model, feature_names, no_top_words):
    '''
    берет модель топик-моделинга, слова и количество топиков, возврашает темы
    '''
    with open('topics_period1_lemm.txt', 'w', encoding='utf-8') as f:
        for topic_idx, topic in enumerate(model.components_):
            print(f"Тема {topic_idx+1}:", file=f)
            topic_words = ", ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]])
            print(topic_words, file=f)

In [50]:
new_stoplist = stoplist2+names_list

In [51]:
def main():
    tfidf_vectorizer = TfidfVectorizer(max_df=0.1, min_df=10, ngram_range=(2, 2), stop_words=new_stoplist)
    tfidf = tfidf_vectorizer.fit_transform(period1_corrected)
    tfidf_feature_names = tfidf_vectorizer.get_feature_names()
    nmf = NMF(n_components=20, random_state=42, alpha=.1, l1_ratio=.5, init='nndsvd', max_iter=100000).fit(tfidf)
    get_topics(nmf, tfidf_feature_names, 10)

In [52]:
if __name__ == '__main__':
    main()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['франц'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [105]:
import gensim

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [106]:
#чанк для вытаскивания фраз
phrases = gensim.models.Phrases(period2_lemmatized)
p = gensim.models.phrases.Phraser(phrases)
ngrammed_texts = p [period2_lemmatized]

In [107]:
gensim_dictionary_for_TM = gensim.corpora.Dictionary(period1_lemmatized)

In [133]:
gensim_dictionary_for_TM.filter_extremes(no_above=0.9, no_below=20)

In [134]:
gensim_dictionary_for_TM.compactify()

In [135]:
corpus = [gensim_dictionary_for_TM.doc2bow(text) for text in period1_lemmatized]

In [136]:
lda = gensim.models.LdaMulticore(corpus, 
                                 num_topics = 20, # число топиков
                                 id2word=gensim_dictionary_for_TM, 
                                 passes=5)

In [137]:
lda.print_topics()

[(0,
  '0.026*"решиться" + 0.025*"театр" + 0.023*"понравиться" + 0.022*"весёлый" + 0.021*"испытывать" + 0.019*"мужчина" + 0.018*"дверь" + 0.017*"проводить" + 0.017*"постоянно" + 0.016*"улица"'),
 (1,
  '0.028*"небо" + 0.026*"природа" + 0.025*"солнце" + 0.022*"искусство" + 0.019*"деревня" + 0.017*"слава" + 0.015*"красота" + 0.014*"грудь" + 0.014*"окно" + 0.014*"работать"'),
 (2,
  '0.048*"е." + 0.022*"совесть" + 0.019*"свобода" + 0.019*"добро" + 0.018*"свободный" + 0.017*"сознание" + 0.015*"духовный" + 0.014*"мальчик" + 0.012*"относиться" + 0.011*"век"'),
 (3,
  '0.086*"государь" + 0.023*"россия" + 0.022*"надобный" + 0.018*"роль" + 0.014*"действовать" + 0.012*"божий" + 0.011*"достойный" + 0.011*"исполнить" + 0.011*"власть" + 0.011*"богатый"'),
 (4,
  '0.028*"существо" + 0.026*"тяжело" + 0.024*"маша" + 0.019*"заставлять" + 0.019*"виноватый" + 0.016*"мама" + 0.016*"искусство" + 0.014*"страдать" + 0.013*"вперёд" + 0.012*"странный"'),
 (5,
  '0.118*"папа" + 0.069*"мама" + 0.049*"маша" + 0.0

In [141]:
#запишем записи нужного периода в словарь чтобы не потерять ID
period3_dict = {}
for key, value in diaries_dic.items():
    for item in period3_ids:
        if key == item:
            period3_dict[key]= value

In [170]:
#пополняем данные записей с паттернами
meta_info_cleaned[(meta_info_cleaned['ID'] == 455569)]

,ID,Автор,ID дневника,Дата
6082,455569,#2281 Лидия Ильинична Яшина,2291,1939-01-08


In [100]:
#ищем записи Галины Ларской
larskaya_ids = []
meta_info_cleaned[meta_info_cleaned['Автор'].str.contains('Ларская', regex=True)]["ID"]:
  #  larskaya_ids.append(item)

SyntaxError: invalid syntax (<ipython-input-100-3a268e7a8a20>, line 3)

In [110]:
for item in meta_info_cleaned[meta_info_cleaned['ID дневника'] == 991]["ID"]:
    larskaya_ids.append(item)

In [113]:
larskaya_records = []
for item in diaries_dic.items():
    for x in larskaya_ids:
        if item[0] == x:
            larskaya_records.append(item)
       #female_records.append(item)  

In [115]:
#larskaya_records

In [167]:
#пробуем искать паттерны с помощью регулярок
for item in period3_dict.items():
    #if re.search(r"настоящ.{2,9}\sлюб.{2,5}", str(item[1])):
    #if re.search(r"(разо(йтись|шлись|шлась|шелся|йдемся)|расста(ться|немся|нусь|лись|ешься)|расставани.{1,2})", str(item[1])):
    #if re.search(r"(не\sмогу|не\sспособен|не\sспособен)\s[а-я]+?любить", str(item[1])):
    #if re.search(r"(в?люб(ить|лю|ил|ила|иться)(сь)?\sпо-настоящему)|(по-настоящему\sв?люб(ить|лю|ил|ила|иться)(сь)?)", str(item[1])):
    #if re.search(r"трепет(но|ный|ного|ать|ал)?.{1,2}", str(item)):
    #if re.search(r"(за)?(при)?ревн(овать|овал|ревную|овала|ревновав)", str(item)):
    if re.search(r"любимого человека", str(item)):
        print(item)

(256806, ['Сегодня большое событие в Алькиной жизни, поэтому и вообще в нашей. Она отдала свою «девичью честь». . . М. Бернесу. Это поразило нас всех, как молния, как гром среди ясного неба, как не знаю что. Такой далекий, знакомый нам только по кино. И вдруг. . . А дело вот в чем. Она случайно узнала, что приехал Бернес к нам в Ленинград. Это она тщательно скрыла от нас. Позвонила ему в гостиницу, пришла к нему (по его приглашению) и в первый же день. . . отдалась. Причем она в него даже не влюблена как следует. Просто он чудный, и она поступила согласно своим воззрениям на этот счет: наслаждение, удовлетворение чувственности — прежде всего в этой жизни, т. к. последняя весьма кратковременна. Кроме того, ей в 20 лет надоело быть девушкой. Не знаю, но сейчас, когда я все это знаю, мне как-то не по себе. Не понравился мне в данном случае Алькин цинизм, более того — он мне чужд и глубоко неприятен. Я не считаю себя чистой, целомудренной и т. д. , я тоже считаю, что любовь к мужчине — гла

In [138]:
#period5_dict